### Ingest circuits.csv file

- #### Step-1 Read the CSV file using the spark dataframe reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [0]:
circuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                     StructField("circuitRef", StringType(), True),
                                     StructField("name", StringType(), True),
                                     StructField("location", StringType(), True),
                                     StructField("country", StringType(), True),
                                     StructField("lat", DoubleType(), True),
                                     StructField("lng", DoubleType(), True),
                                     StructField("alt", IntegerType(), True),
                                     StructField("url", StringType(), True)
])

In [0]:
# ⚠️ L'utilisation de `inferSchema=True` est lente et inadaptée à la production.
#    Spark doit analyser toutes les données pour deviner les types, ce qui ralentit le chargement.

circuits_df = spark.read \
.option("header",True) \
.schema(circuits_schema) \
.csv("dbfs:/mnt/f1dlportfolio/raw/circuits.csv")

In [0]:
display(circuits_df.printSchema())

In [0]:
circuits_df.describe().show()

In [0]:
circuits_df.show(truncate=False)

In [0]:
display(circuits_df)

- #### Step 2 - Select only the required columns

In [0]:
circuits_selected_df = circuits_df.select("circuitId", "circuitRef", "name", "location", "country", "lat", "lng", "alt")

In [0]:
circuits_selected_df = circuits_df.select(circuits_df.circuitId, circuits_df.circuitRef, circuits_df.name, circuits_df.location, circuits_df.country, circuits_df.lat, circuits_df.lng, circuits_df.alt)

In [0]:
circuits_selected_df = circuits_df.select(circuits_df["circuitId"], circuits_df["circuitRef"], circuits_df["name"], circuits_df["location"], circuits_df["country"], circuits_df["lat"], circuits_df["lng"], circuits_df["alt"])

In [0]:
from pyspark.sql.functions import col	

In [0]:
circuits_selected_df = circuits_df.select(col("circuitId"), col("circuitRef"), col("name"), col("location"), col("country"), col("lat"), col("lng"), col("alt"))

In [0]:
display(circuits_selected_df)

- #### Step 3 - Rename the column as required

In [0]:
circuits_rename_df = circuits_selected_df.withColumnRenamed("circuitId", "circuit_id") \
    .withColumnRenamed("circuitRef", "circuit_ref") \
    .withColumnRenamed("lat", "latitude") \
    .withColumnRenamed("lng", "longitude") \
    .withColumnRenamed("alt", "altitude")

In [0]:
display(circuits_rename_df)

- #### Step 4 - Add ingestion date to the dataframe

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
circuit_final_df = circuits_rename_df.withColumn("ingestion_date", current_timestamp())

In [0]:
display(circuit_final_df)

In [0]:
circuit_final_df.write.mode("overwrite").parquet("/mnt/f1dlportfolio/processed/circuits")

In [0]:
%fs
ls /mnt/f1dlportfolio/processed/circuits

In [0]:
df = spark.read.parquet("/mnt/f1dlportfolio/processed/circuits")

In [0]:
display(df)